In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the data
df = pd.read_csv("cs-training.csv")

In [2]:
# Pre-processing
df.fillna(df.mean(), inplace=True)
df = df[df.MonthlyIncome < df.MonthlyIncome.quantile(0.95)]
df = df[df.NumberOfDependents < df.NumberOfDependents.quantile(0.95)]

In [3]:
df.columns

Index(['Unnamed: 0', 'SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

In [4]:
# Feature selection
features = ["RevolvingUtilizationOfUnsecuredLines", "age", "NumberOfTime30-59DaysPastDueNotWorse",
            "DebtRatio", "MonthlyIncome", "NumberOfOpenCreditLinesAndLoans",
            "NumberOfTimes90DaysLate", "NumberRealEstateLoansOrLines", "NumberOfTime60-89DaysPastDueNotWorse",
            "NumberOfDependents"]
target = "SeriousDlqin2yrs"

In [5]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

In [7]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     24341
           1       0.51      0.18      0.26      1740

    accuracy                           0.93     26081
   macro avg       0.73      0.58      0.61     26081
weighted avg       0.91      0.93      0.92     26081



In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

# Define the hyperparameters to search over
param_grid = {'n_estimators': [200,300], 'max_depth': [None,20]}

In [9]:
# Define the scoring metric (in this case, recall for the positive class)
recall_scorer = make_scorer(recall_score, pos_label=1)

# Create a Random Forest Classifier
clf = RandomForestClassifier(random_state=42,class_weight='balanced')

# Create the grid search object
grid_search = GridSearchCV(clf, param_grid, scoring=recall_scorer, cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'max_depth': [None, 20], 'n_estimators': [200, 300]},
             scoring=make_scorer(recall_score, pos_label=1))

In [10]:
# Get the best parameters
best_params = grid_search.best_params_
print('Best parameters:', best_params)

# Get the best model
best_model = grid_search.best_estimator_

Best parameters: {'max_depth': 20, 'n_estimators': 300}


In [13]:
# Train the Random Forest Classifier on the training data
clf = RandomForestClassifier(random_state=42,max_depth=20,n_estimators=300,class_weight='balanced')
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=300,
                       random_state=42)

In [15]:
# Make predictions on the testing data
y_pred = clf.predict(X_test)

In [16]:
# Evaluate the model's performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     24341
           1       0.39      0.30      0.34      1740

    accuracy                           0.92     26081
   macro avg       0.67      0.63      0.65     26081
weighted avg       0.91      0.92      0.92     26081

